In [ ]:
from pathlib import Path

import torch

from ssl_brainmet.models.build_nnunet_model import build_model_from_plans

In [2]:
project_dir = Path(".").resolve().parents[0]

model_trained = build_model_from_plans(
    project_dir / "ssl_brainmet/config/nnUNetPlans.json",
    project_dir / "ssl_brainmet/config/dataset.json",
    configuration="3d_fullres",
    deep_supervision=True,
)

In [3]:
weights_path = "/home/valentin/data/target/data/nnUnet_results/Dataset511_BrainMetsSegT1to3/nnUNetTrainerFineTuning__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth"

In [ ]:
state_dict = torch.load(weights_path)

In [ ]:
state_dict

In [ ]:
model_trained.load_state_dict(state_dict["network_weights"])

In [6]:
weights_path = "/home/vincent/repos/ssl-bm/weights/cnn3d_nnunet_firstgood.pt"

model_ssl = build_model_from_plans(
    project_dir / "ssl_brainmet/config/nnUNetPlans.json",
    project_dir / "ssl_brainmet/config/dataset.json",
    configuration="3d_fullres",
    deep_supervision=True,
)

In [ ]:

state_dict = torch.load(weights_path)
# Filter the state dict to only keep weights for the encoder
encoder_state_dict = {k[len("encoder."):]: v for k, v in state_dict.items() if k.startswith("encoder.")}

# Load the filtered weights into the encoder
model_ssl.encoder.load_state_dict(encoder_state_dict)

In [ ]:
encoder1_state = model_trained.encoder.state_dict()
encoder2_state = model_ssl.encoder.state_dict()

# Check if state_dicts are identical
if all(torch.equal(encoder1_state[key], encoder2_state[key]) for key in encoder1_state):
    print("The encoders have identical weights.")
else:
    print("The encoders have different weights.")

In [ ]:
def compute_encoder_differences(model1, model2):
    """
    Compute the max absolute difference between encoder weights of two models.

    Args:
        model1: First model with an encoder.
        model2: Second model with an encoder.

    Returns:
        dict: Dictionary containing max differences for each layer.
    """
    state_dict1 = model1.encoder.state_dict()
    state_dict2 = model2.encoder.state_dict()
    differences = {}

    for key in state_dict1.keys():
        diff = torch.abs(state_dict1[key] - state_dict2[key]).max().item()
        differences[key] = diff

    return differences

# Usage example:
encoder_differences = compute_encoder_differences(model_trained, model_ssl)
for layer, diff in encoder_differences.items():
    print(f"Max difference in {layer}: {diff}")
